In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

scores_tns = '(DESCRIPTION = (ADDRESS = (PROTOCOL = TCP)(HOST = 172.16.1.16)(PORT = 1521)) '
scores_tns += '(CONNECT_DATA = (SERVER = DEDICATED) (SERVICE_NAME = crcpdb01) ) )'
scores_con = create_engine("oracle+cx_oracle://scores:scores@" + scores_tns)

In [ ]:
def drop_table(table_name):
    return '''
    drop table {}
    '''.format(table_name)

In [ ]:
def count_table(table_name):
    return '''
    select count (*) from {}
    '''.format(table_name)

In [ ]:
ff_fico_00a00 = ("cb_CONS_CREDIT_hst", """
CREATE TABLE cb_CONS_CREDIT_hst
AS
  SELECT DISTINCT
    to_char(HSTF.UNIQUE_ROOT_ID)                                                                UNIQUE_ROOT_ID,
    to_char(HSTF.primary_ROOT_ID)                                                               primary_ROOT_ID,
    TRUNC(MONTHS_BETWEEN(LAST_DAY(SCAL.DATE_REPORTED), LAST_DAY(HSTF.DATE_REPORTED)))           MONTH_INDEX,
    TO_CHAR(SCAL.DATE_REPORTED, 'YYYY-MM-DD')                                                   DATE_REPORTED,
    TO_CHAR(HSTF.DATE_REPORTED, 'YYYY-MM-DD')                                                   HST_DATE_REPORTED,
    GREATEST(HSTF.MAX_NUM_DAYS_DUE, HSTF.MAX_NUM_DAYS_DUE_INT)                                  NDOD,
    ROUND(ABS(HSTF.OVER_DUE_AMT_INT + HSTF.OVER_DUE_AMT_PRICIPAL) * NVL(ER.SELLING_RATE, 1), 2) OVERDUE_AMT,
    ROUND(ABS(HSTF.CURRENT_BAL) * NVL(ER.SELLING_RATE, 1), 2)                                   BALANCE,
    hstf.file_id
  FROM sbhistory.HST_CONS_CREDIT_FACILITY@sb2a12 HSTF,
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_cons_account_lookup@sb2a12 a
     )
     WHERE rnk = 1
    ) SCAL,
    sbapi.AA_FICO_EXRATE@sb2a12 ER
  WHERE ER.CURR_CODE (+) = HSTF.CURRENCY
        AND HSTF.DATE_REPORTED BETWEEN ADD_MONTHS(SCAL.DATE_REPORTED, -23) AND SCAL.DATE_REPORTED
        AND HSTF.DATE_REPORTED <> SCAL.DATE_REPORTED
        AND HSTF.UNIQUE_ROOT_ID = SCAL.UNIQUE_ROOT_ID
        AND SCAL.ACC_STATUS <> '002'
""")

In [ ]:
ff_fico_00a01 = ("mf_CONS_CREDIT_hst", """
CREATE TABLE mf_CONS_CREDIT_hst
AS
  SELECT DISTINCT
    to_char(HSTF.UNIQUE_ROOT_ID)                                                      UNIQUE_ROOT_ID,
    to_char(HSTF.primary_ROOT_ID)                                                     primary_ROOT_ID,
    TRUNC(MONTHS_BETWEEN(LAST_DAY(SCAL.DATE_REPORTED), LAST_DAY(HSTF.DATE_REPORTED))) MONTH_INDEX,
    TO_CHAR(SCAL.DATE_REPORTED, 'YYYY-MM-DD')                                         DATE_REPORTED,
    TO_CHAR(HSTF.DATE_REPORTED, 'YYYY-MM-DD')                                         HST_DATE_REPORTED,
    GREATEST(HSTF.MAX_NUM_DAYS_DUE)                                                   NDOD,
    ROUND(ABS(HSTF.OVER_DUE_AMT_PRICIPAL) * NVL(ER.SELLING_RATE, 1), 2)               OVERDUE_AMT,
    ROUND(ABS(HSTF.CURRENT_BAL) * NVL(ER.SELLING_RATE, 1), 2)                         BALANCE,
    hstf.file_id
  FROM sbhistory.HST_MFCONS_CREDIT_FACILITY@sb2a12 HSTF,
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_mfcons_account_lookup@sb2a12 a
     )
     WHERE rnk = 1
    ) SCAL,
    sbapi.AA_FICO_EXRATE@sb2a12 ER
  WHERE ER.CURR_CODE (+) = HSTF.CURRENCY
        AND HSTF.DATE_REPORTED 
            BETWEEN ADD_MONTHS(SCAL.DATE_REPORTED, -23) AND SCAL.DATE_REPORTED
        AND HSTF.DATE_REPORTED <> SCAL.DATE_REPORTED
        AND HSTF.UNIQUE_ROOT_ID = SCAL.UNIQUE_ROOT_ID
        --         AND HSTF.SIN = SCAL.SIN
        AND SCAL.ACC_STATUS <> '002'
""")

In [ ]:
ff_fico_00a02 = ("mg_CONS_CREDIT_hst", """
CREATE TABLE mg_CONS_CREDIT_hst
AS
  SELECT DISTINCT
    to_char(HSTF.UNIQUE_ROOT_ID)                                                                    UNIQUE_ROOT_ID,
    to_char(HSTF.primary_ROOT_ID)                                                                   primary_ROOT_ID,
    TRUNC(MONTHS_BETWEEN(LAST_DAY(SCAL.DATE_REPORTED), LAST_DAY(HSTF.DATE_REPORTED)))               MONTH_INDEX,
    TO_CHAR(SCAL.DATE_REPORTED, 'YYYY-MM-DD')                                                       DATE_REPORTED,
    TO_CHAR(HSTF.DATE_REPORTED, 'YYYY-MM-DD')                                                       HST_DATE_REPORTED,
    GREATEST(HSTF.DAYS_OVERDUE_PRINCIPAL, HSTF.DAYS_OVERDUE_INTEREST)                               NDOD,
    ROUND(ABS(HSTF.AMT_OVERDUE_INTEREST + HSTF.AMT_OVERDUE_PRINCIPAL) * NVL(ER.SELLING_RATE, 1), 2) OVERDUE_AMT,
    ROUND(ABS(HSTF.ACCOUNT_BALANCE) * NVL(ER.SELLING_RATE, 1), 2)                                   BALANCE,
    hstf.file_id
  FROM sbhistory.HST_MGCONS_FACILITY@sb2a12 HSTF,
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_mgcons_account_lookup@sb2a12 a
     )
     WHERE rnk = 1
    ) SCAL,
    sbapi.AA_FICO_EXRATE@sb2a12 ER
  WHERE ER.CURR_CODE (+) = HSTF.CURRENCY
        AND HSTF.DATE_REPORTED 
            BETWEEN ADD_MONTHS(SCAL.DATE_REPORTED, -23) AND SCAL.DATE_REPORTED
        AND HSTF.DATE_REPORTED <> SCAL.DATE_REPORTED
        AND HSTF.UNIQUE_ROOT_ID = SCAL.UNIQUE_ROOT_ID
        --         AND HSTF.SIN = SCAL.SIN
        AND SCAL.ACCount_STATUS <> '002'
""")

In [ ]:
ff_fico_00a03 = ("cb_data_block_builds", """
CREATE TABLE cb_data_block_builds AS
  SELECT DISTINCT
    'CM'                     segment,
    cm.merged_ruid,
    scal.unique_root_id,
    scal.primary_root_id,
    currency,
    scal.data_prdr_id,
    first_disburse_date,
    sanction_date,
    sanction_amt,
    installment_amount_int,
    installment_amount,
    reason_for_closure,
    loan_type,
    number_of_installments,
    no_of_installments_int,
    repayment_cycle,
    current_bal,
    last_amount_paid,
    last_amount_paid_int,
    over_due_amt_pricipal,
    over_due_amt_int,
    max_num_days_due,
    max_num_days_due_int,
    asset_classification,
    acc_status,
    scal.date_reported,
    date_latest_pay_received,
    date_latest_pay_received_int,
    date_acc_close,
    dispute_id,
    cast('' AS VARCHAR2(10)) dispute_status,
    --    scal.sin,
    cm.date_of_birth,
    cm.gender,
    cm.sub_nationality, scal.file_id
  FROM (SELECT *
        FROM (
          SELECT
            a.*,
            rank()
            OVER (PARTITION BY unique_root_id
              ORDER BY date_reported DESC) rnk
          FROM sbinternal.sb_cons_account_lookup@sb2SUB12 a
        )
        WHERE rnk = 1
       ) scal,
    sbinternal.consumer_master@sb2SUB12 cm,
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_cons_subject_lookup@sb2SUB12 a
     )
     WHERE rnk = 1
    ) scsl
  WHERE scsl.isdelinked IS NULL
        AND (cm.ruid = scsl.ruid OR cm.merged_ruid = scsl.ruid)
        AND scal.unique_root_id = scsl.unique_root_id
""")

In [ ]:
ff_fico_00a04 = ("mf_data_block_builds", """
CREATE TABLE mf_data_block_builds AS
  SELECT DISTINCT
    'MF'                     segment,
    cm.merged_ruid,
    scal.unique_root_id,
    scal.primary_root_id,
    currency,
    scal.data_prdr_id,
    first_disburse_date,
    first_disburse_date      sanction_date,
    sanction_amt,
    to_number('0')           installment_amount_int,
    installment_amount,
    reason_for_closure,
    loan_type,
    to_number('0')           number_of_installments,
    to_number('0')           no_of_installments_int,
    repayment_cycle,
    current_bal,
    last_amount_paid,
    to_number('0')           last_amount_paid_int,
    over_due_amt_pricipal,
    to_number('0')           over_due_amt_int,
    max_num_days_due,
    to_number('0')           max_num_days_due_int,
    asset_classification,
    acc_status,
    scal.date_reported,
    date_latest_pay_received,
    cast(NULL AS DATE)       date_latest_pay_received_int,
    date_acc_close,
    dispute_id,
    cast('' AS VARCHAR2(10)) dispute_status,
    --    scal.sin,
    cm.date_of_birth,
    cm.gender,
    cm.sub_nationality, scal.file_id
  FROM
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_mfcons_account_lookup@sb2SUB12 a
     )
     WHERE rnk = 1
    ) scal,
    sbinternal.consumer_master@sb2SUB12 cm,
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_mfcons_subject_lookup@sb2SUB12 a
     )
     WHERE rnk = 1
    ) scsl
  WHERE scsl.isdelinked IS NULL
        AND (cm.ruid = scsl.ruid OR cm.merged_ruid = scsl.ruid)
        AND scal.unique_root_id = scsl.unique_root_id
""")

In [ ]:
ff_fico_00a05 = ("mg_data_block_builds", """
CREATE TABLE mg_data_block_builds AS
  SELECT DISTINCT
    'MG'                       segment,
    cm.merged_ruid,
    scal.unique_root_id,
    scal.primary_root_id,
    currency,
    scal.data_prdr_id,
    first_disbursment_date     first_disburse_date,
    approval_date              sanction_date,
    amount_approved            sanction_amt,
    instl_amount_interest      installment_amount_int,
    instl_amount_principal     installment_amount,
    reason_acc_closure         reason_for_closure,
    facility_type              loan_type,
    to_number('0')             number_of_installments,
    to_number('0')             no_of_installments_int,
    repay_freq_principal       repayment_cycle,
    account_balance            current_bal,
    last_principal_amount      last_amount_paid,
    last_interest_amount       last_amount_paid_int,
    amt_overdue_principal      over_due_amt_pricipal,
    amt_overdue_interest       over_due_amt_int,
    days_overdue_principal     max_num_days_due,
    days_overdue_interest      max_num_days_due_int,
    asset_classification,
    account_status             acc_status,
    scal.date_reported,
    pymt_date_principal        date_latest_pay_received,
    pymt_date_interest         date_latest_pay_received_int,
    account_closing_date       date_acc_close,
    dispute_id,
    cast(NULL AS VARCHAR2(10)) dispute_status,
    --    scal.sin,
    cm.date_of_birth,
    cm.gender,
    cm.sub_nationality, scal.file_id
  FROM
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_mgcons_account_lookup@sb2SUB12 a
     )
     WHERE rnk = 1
    ) scal,
    sbinternal.consumer_master@sb2SUB12 cm,
    (SELECT *
     FROM (
       SELECT
         a.*,
         rank()
         OVER (PARTITION BY unique_root_id
           ORDER BY date_reported DESC) rnk
       FROM sbinternal.sb_mgcons_subject_lookup@sb2SUB12 a
     )
     WHERE rnk = 1
    ) scsl
  WHERE scsl.isdelinked IS NULL
        AND (cm.ruid = scsl.ruid OR cm.merged_ruid = scsl.ruid)
        AND scal.unique_root_id = scsl.unique_root_id
""")

In [ ]:
ff_fico_00a06 = ("institution_master", """
CREATE TABLE institution_master AS
  SELECT *
  FROM sbapi.sb_institution_master@sb2SUB12
""")

In [ ]:
ff_fico_00a07 = ("aa_fico_exrate","""
CREATE TABLE aa_fico_exrate AS
  SELECT *
  from crcops.exchange_rate
""")

In [ ]:
ff_fico_00a08 = ("all_enquiries", """
CREATE TABLE all_enquiries AS
  SELECT DISTINCT
    CM.MERGED_RUID,
    'INQC|'
    || TO_CHAR(SI.REQUEST_DATE, 'YYYY-MM-DD')
    || '|'
    || PRODUCT_TYPE
    || '|'
    || '' enquiry_block
  FROM CONSUMER_MASTER@sb2SUB12 CM,
    sbapi.SB_INQUIRY@sb2SUB12 SI,
    sbapi.SB_REPORT_DETAILS@sb2SUB12 SRD
  WHERE SRD.REQUEST_ID = SI.REQUEST_ID
        AND TO_CHAR(SI.REQUEST_DATE, 'YYYYMM') <= TO_CHAR(SYSDATE, 'YYYYMM')
        AND CM.RUID = SRD.RUID
""")

In [ ]:
ff_fico_00a08a = ("fico_enquiries_block", """
create table fico_enquiries_block as
select merged_ruid,
listagg(enquiry_block, '|') within group (order by enquiry_block) enquiries_block
from aa_fico_enquiry_blocks group by merged_ruid
""")

In [ ]:
ff_fico_01a = ("all_cons_credit_hst", """
CREATE TABLE all_cons_credit_hst
AS SELECT *
   FROM cb_CONS_CREDIT_hst
   UNION ALL SELECT *
             FROM mf_CONS_CREDIT_hst
   UNION ALL SELECT *
             FROM mg_CONS_CREDIT_hst
""")

In [ ]:
ff_fico_01b00 = ("months_index_exceptions", """
CREATE TABLE months_index_exceptions
AS
  SELECT
    unique_root_id,
    COUNT(*) counts
  FROM all_cons_credit_hst
  HAVING COUNT(*) > 23
  GROUP BY unique_root_id
""")

In [ ]:
ff_fico_01b01 = ("months_index_exceptions", """
CREATE TABLE monthly_index_exceptions
AS
  SELECT
    unique_root_id,
    month_index,
    COUNT(month_index) counts
  FROM all_cons_credit_hst
  HAVING COUNT(month_index) > 1
  GROUP BY unique_root_id, month_index
""")

In [ ]:
ff_fico_01b01 = ("", """
create table hst12cons0credit as 
SELECT MAX(intb.file_id) max_file_id, intb.unique_root_id,
    TO_CHAR(LAST_DAY(TO_DATE(intb.HST_DATE_REPORTED, 'yyyy-mm-dd')), 'yyyy-mm-dd') last_day_month_reported
FROM all_cons_credit_hst intb 
group by intb.unique_root_id, TO_CHAR(LAST_DAY(TO_DATE(intb.HST_DATE_REPORTED, 'yyyy-mm-dd')), 'yyyy-mm-dd')
""")